# Attack viz

In [2]:
import matplotlib
import pandas as pd
import pymysql
%matplotlib inline

In [3]:
connection = pymysql.connect(host='localhost',
                             user='fingerpatch',
                             password='fingerpatch',
                             db='fingerpatch',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [4]:
attack_table = pd.read_sql("SELECT * FROM `ubuntu_captures` ",connection)
ground_truth = pd.read_sql("SELECT * FROM `ubuntu_packets` ",connection)
target = attack_table.loc[0]

In [11]:
ground_truth.count()[0]

128148

In [6]:
target

capture_id                                     1
nb_flows                                       3
Flow1               target->danava.canonical.com
Flow2               danava.canonical.com->target
Flow3               target->danava.canonical.com
Flow4                                       None
Flow5                                       None
nb_Payload_send1                               0
nb_Payload_send2                           67874
nb_Payload_send3                             173
nb_Payload_send4                            None
nb_Payload_send5                            None
Name: 0, dtype: object

## Mounting an Attack for matching a specific capture to a package.

## Dependens discovery

let's take only the packages that have only one depends.

Sort them by ascending size

In [55]:
tmp = ground_truth
tmp["#Depends"] = tmp["Depends"].map(lambda x: 0 if x == "" else len(x.split(",")))
two_dep_first10 = tmp[tmp["#Depends"] == 1].sort_values(by = "Size", ascending=True)[:10]

In [64]:
two_dep_first10.iloc[0]

capture_id                                                         1
Package                                                      readpst
Version                                               0.6.59-1build1
Architecture                                                     all
Size                                                             796
Installed-Size                                                    21
Priority                                                    optional
Maintainer         Ubuntu Developers <ubuntu-devel-discuss@lists....
SHA1                        da3b4b8e1da9e72c3f8a82d03fd620a8590c7186
Description            Converts Outlook PST files to mbox and others
parsedFrom         packages/archive.ubuntu.com_ubuntu_dists_trust...
Description-md5                     3ef2b339ca5bf4fb7de6354b65f7cf59
Bugs                      https://bugs.launchpad.net/ubuntu/+filebug
Origin                                                        Ubuntu
MD5sum                            

Seeking about that depends

In [63]:
ground_truth[ground_truth["Package"] == two_dep_first10.iloc[0]["Depends"]]

,capture_id,Package,Version,Architecture,Size,Installed-Size,Priority,Maintainer,SHA1,Description,...,MD5sum,Depends,Homepage,Source,SHA256,Section,Supported,Filename,packageMode,#Depends
18279,1,pst-utils,0.6.59-1build1,amd64,62092,181,optional,Ubuntu Developers <ubuntu-devel-discuss@lists....,7e5afc4156a87f8c02ae369428ffcbb967111bed,tools for reading Microsoft Outlook PST files,...,f75684e6ddb8a1a651a629de9bd42d5d,"libc6 (>= 2.14), libgcc1 (>= 1:4.1.1), libgd3 ...",http://www.five-ten-sg.com/libpst/,libpst,807a10cc7b8062403018d37a6d19cd210bd9f836ae62d9...,utils,9m,pool/main/libp/libpst/pst-utils_0.6.59-1build1...,releases,7
82353,1,pst-utils,0.6.59-1build1,amd64,62092,181,optional,Ubuntu Developers <ubuntu-devel-discuss@lists....,7e5afc4156a87f8c02ae369428ffcbb967111bed,tools for reading Microsoft Outlook PST files,...,f75684e6ddb8a1a651a629de9bd42d5d,"libc6 (>= 2.14), libgcc1 (>= 1:4.1.1), libgd3 ...",http://www.five-ten-sg.com/libpst/,libpst,807a10cc7b8062403018d37a6d19cd210bd9f836ae62d9...,utils,9m,pool/main/libp/libpst/pst-utils_0.6.59-1build1...,source,7


In [46]:
two_dep_first10[["Package","Version","Architecture", "Size"]].to_csv(path_or_buf="two_dep_first10")

In [8]:
extra_size = 283
variation = 5

In [10]:
out = pd.read_csv('/home/alex/Documents/semesterProject/fingerpatch/crawl/sizes.out', names=['size'],header=None)

In [11]:
len(out['size'].unique())/len(out)

0.7668824674982894

In [12]:
ground_truth['Size'].isnull().any()

False

In [23]:
total_len =len(out)
out['count_size'] = ""
out['#'] = '#'
tmp = out.groupby("size").count()
tmp = tmp.groupby("count_size").count()
total_len = tmp["#"].sum()
tmp['#'] = tmp['#'].apply(lambda x: x/total_len)

In [ ]:
total_len =len(out)
out['count_size'] = ""
out['#'] = '#'
tmp = out.groupby("size").count()
tmp = tmp.groupby("count_size").count()
total_len = tmp["#"].sum()
tmp['#'] = tmp['#'].apply(lambda x: x/total_len)
tmp

In [ ]:
d['test'] = "t"
s = d.groupby('size')
d

In [24]:
d = {'size' : [1, 2, 4, 1 , 4 , 2, 4, 1, 5,4 ,6]}
d = pd.DataFrame(data=d)
d['test'] = "t"
d["other"] = "d"
s = d.groupby('size')
d

,size,test,other
0,1,t,d
1,2,t,d
2,4,t,d
3,1,t,d
4,4,t,d
5,2,t,d
6,4,t,d
7,1,t,d
8,5,t,d
9,4,t,d


In [36]:
def possible_match(x, size_to_match, average_extra_size ,variation):
    if(average_extra_size - variation > size_to_match and average_extra_size +variation<size_to_match):
        return True
    else:
        return False
    


In [37]:
possible_match(32, 32 ,32)

False

In [54]:
eval("['target->keeton.canonical.com', 'keeton.canonical.com->target', 'target->keeton.canonical.com', 'target->economy.canonical.com', 'target->economy.canonical.com', 'economy.canonical.com->target', 'economy.canonical.com->target', 'target->economy.canonical.com', 'economy.canonical.com->target', 'target->economy.canonical.com', 'target->economy.canonical.com', 'economy.canonical.com->target', 'target->economy.canonical.com']")

['target->keeton.canonical.com',
 'keeton.canonical.com->target',
 'target->keeton.canonical.com',
 'target->economy.canonical.com',
 'target->economy.canonical.com',
 'economy.canonical.com->target',
 'economy.canonical.com->target',
 'target->economy.canonical.com',
 'economy.canonical.com->target',
 'target->economy.canonical.com',
 'target->economy.canonical.com',
 'economy.canonical.com->target',
 'target->economy.canonical.com']

attack